In [1]:
import requests
import pandas as pd
from datetime import datetime

def get_binance_data(symbol, interval, start, end):
    url = 'https://api.binance.com/api/v3/klines'
    all_data = pd.DataFrame()

    while start < end:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start,
            'limit': 1000  # Max limit
        }
        response = requests.get(url, params=params)
        data = response.json()

        # Convert to DataFrame and add Symbol column
        columns = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 
                   'Close time', 'Quote asset volume', 'Number of trades', 
                   'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
        df = pd.DataFrame(data, columns=columns)
        df['Symbol'] = symbol

        # Convert timestamps to datetime
        df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
        df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')

        all_data = pd.concat([all_data, df], ignore_index=True)

        # Update start time for next loop
        start = df['Close time'].max().value // 10**6  # Convert to timestamp in milliseconds

    return all_data

# Define your cryptocurrencies and time period
crypto_symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'XRPUSDT', 'ADAUSDT']  # Example symbols
start_date = int(datetime(2018, 5, 1).timestamp() * 1000)  # Start date (common range start)
end_date = int(datetime.now().timestamp() * 1000)  # End date (current date)

# Fetch data for each cryptocurrency
all_data = pd.DataFrame()
for symbol in crypto_symbols:
    crypto_data = get_binance_data(symbol, '1d', start_date, end_date)
    all_data = pd.concat([all_data, crypto_data], ignore_index=True)

# Reorder columns to put 'Symbol' first
column_order = ['Symbol'] + [col for col in all_data.columns if col != 'Symbol']
all_data = all_data[column_order]

# Display the combined data
all_data.head()


,Symbol,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,BTCUSDT,2018-05-02,9071.48000000,9268.00000000,8970.20000000,9247.84000000,26123.54396100,2018-05-02 23:59:59.999,238421833.75811328,188258,13960.45552800,127439644.78329857,0
1,BTCUSDT,2018-05-03,9247.81000000,9844.00000000,9168.40000000,9750.00000000,38768.38828800,2018-05-03 23:59:59.999,366028365.22714452,282581,21093.14034300,199234352.40239446,0
2,BTCUSDT,2018-05-04,9750.00000000,9830.04000000,9520.85000000,9713.99000000,28681.58887900,2018-05-04 23:59:59.999,277565997.63316434,210165,14779.49955400,143070410.02237806,0
3,BTCUSDT,2018-05-05,9714.00000000,10020.00000000,9682.00000000,9864.00000000,24990.01834500,2018-05-05 23:59:59.999,246807563.50646492,212076,13011.84429300,128527642.99648472,0
4,BTCUSDT,2018-05-06,9863.99000000,9970.00000000,9417.03000000,9659.01000000,27690.35155900,2018-05-06 23:59:59.999,267359420.30359016,229309,13297.50695900,128427770.52523223,0
